# Molecular Toxicity Prediction (Tox21) using GIN

In this notebook, we implement and train a Graph Isomorphism Network (GIN) for predicting molecular toxicity based on the Tox21 dataset. GIN is generally more powerful than GCN for graph representation learning.

## 1. Environment Setup in Colab

Run the following code to install PyTorch Geometric and other dependencies:

In [ ]:
import os
import shutil

# This cell is intended to be run only in Google Colab
if 'COLAB_GPU' in os.environ:
    print("Running in Colab...")

    # 1. Remove existing folder if it already exists
    repo_name = 'gnn-molecule-prediction'
    repo_path = os.path.join('/content', repo_name)

    if os.path.exists(repo_path):
        shutil.rmtree(repo_path)
        print(f"Removed existing directory: {repo_path}")
    else:
        print(f"No existing directory found: {repo_path}")

    # 2. Clone the GitHub repository
    %cd /content
    !git clone https://github.com/sth-s/gnn-molecule-prediction.git

    # 3. Change working directory to the project root
    %cd gnn-molecule-prediction

    # 4. Install Conda support in Colab and restart runtime
    !pip install -q condacolab
    import condacolab
    condacolab.install()  # This will automatically restart the runtime
    !conda env update -n base -f environment.yml
    %cd gnn-molecule-prediction

else:
    os.chdir('../')
    print(f"Changed working directory to: {os.getcwd()}")


## 2. Import Libraries

In [ ]:
# PyTorch and PyG
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GINConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

# Chemistry and data processing
from rdkit import Chem
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluation and splitting
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, roc_curve
from sklearn.model_selection import train_test_split

# Our custom data loader
from src.data_utils import load_tox21
# Import our custom model and training functions
from src.model import GIN
from src.train import train_epoch, evaluate

# Set visualization style
sns.set_theme(style="whitegrid")
sns.set_palette('muted')

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 3. Loading and Preparing Tox21 Data

### 3.1 About the Data Loading Module

We use a custom developed `load_tox21` module from the `src.data_utils` package, which provides the following features:

- **Automatic data downloading**: if the `tox21.csv` file is missing, the module will automatically download it from the official source
- **SMILES to graphs conversion**: each molecule is converted into a graph with node and edge attributes
- **Result caching**: results are saved to a cache to speed up subsequent runs
- **Flexible configuration**: you can specify the data path, file name, target columns, and other parameters

Detailed information about the `load_tox21` function and usage examples are available in the project's README.

In [ ]:
# Load the Tox21 dataset using our custom function
# If the tox21.csv file is missing, it will be automatically downloaded
dataset = load_tox21(
    root="data/Tox21",           # root directory for the data
    filename="tox21.csv",        # name of the CSV file
    smiles_col="smiles",         # column containing SMILES strings
    mol_id_col="mol_id",         # column containing molecule IDs
    cache_file="data.pt",        # name of the cache file
    recreate=False,              # Use existing processed file if available
    auto_download=True           # automatically download if missing
)

print(f"Type of dataset object: {type(dataset)}")
try:
    print(f"dataset._slices: {dataset._slices}")
except AttributeError:
    print(f"dataset has no attribute _slices")
print(f"Total graphs (len(dataset)): {len(dataset)}")

# Check if we can access the first item in the dataset
if len(dataset) > 0:
    print("Attempting to access dataset[0]...")
    first_item = None
    try:
        first_item = dataset[0]
        print(f"dataset[0] type: {type(first_item)}")
        if first_item is not None:
            print(f"Number of node features: {first_item.x.shape[1]}")
            if hasattr(first_item, 'mol_id'):
                 print(f"Mol ID for the first molecule: {first_item.mol_id}")
            else:
                 print(f"Mol ID attribute not found in first_item.")
            
            # Check the presence of y and y_mask
            if hasattr(first_item, 'y'):
                print(f"Task labels (y) shape: {first_item.y.shape}")
                print(f"Task labels (y): {first_item.y}")
            if hasattr(first_item, 'y_mask'):
                print(f"Task mask (y_mask) shape: {first_item.y_mask.shape}")
                print(f"Task mask (y_mask): {first_item.y_mask}")
        else:
            print(f"dataset[0] is None.")
    except Exception as e:
        print(f"Error accessing dataset[0] or its attributes: {e}")
else:
    print("Dataset is empty (len(dataset) is 0).")


### 3.3 Verify mol_id and Dataset Properties (Revised)

In [ ]:
# Verify that mol_id is present and check dataset properties
print("\nRunning second verification cell:")
if len(dataset) > 0:
    data_example = None
    try:
        data_example = dataset[0] # This might be None if the previous cell showed it was None
    except Exception as e:
        print(f"Error getting dataset[0] in second cell: {e}")
    
    if data_example is not None:
        print("Example graph from the dataset (after potential reprocessing):")
        print(data_example)
        
        if hasattr(data_example, 'mol_id'):
            print(f"\nMol ID for the first molecule: {data_example.mol_id}")
        else:
            print("\nmol_id attribute not found in the first molecule (data_example).")
        
        print(f"Edge index dimensions: {data_example.edge_index.shape}")
        print(f"Number of atoms (nodes): {data_example.num_nodes}")
        print(f"Task labels (y): {data_example.y}")
        print(f"Task mask (y_mask): {data_example.y_mask}")
    elif len(dataset) > 0: # dataset[0] was None but dataset is not empty
        print("data_example (dataset[0]) is None, cannot print details.")
else:
    print("Dataset is empty in second verification cell.")

In [ ]:
# Analysis of graph properties in the dataset
if len(dataset) > 0:
    data_example = dataset[0]
    print("Example graph from the dataset:")
    print(data_example)
    print(f"Edge index dimensions: {data_example.edge_index.shape}")
    print(f"Number of atoms (nodes): {data_example.num_nodes}")
    print(f"Task labels (y): {data_example.y}")

    # Dataset statistics
    nodes_count = []
    edges_count = []
    for i in range(min(1000, len(dataset))):
        data = dataset[i]
        nodes_count.append(data.num_nodes)
        edges_count.append(data.edge_index.shape[1])
    
    print(f"\nGraph statistics (based on a sample of {len(nodes_count)} molecules):")
    print(f"Average number of atoms: {np.mean(nodes_count):.2f} ± {np.std(nodes_count):.2f}")
    print(f"Average number of bonds: {np.mean(edges_count)/2:.2f} ± {np.std(edges_count)/2:.2f}")
    print(f"Min/max atoms: {np.min(nodes_count)}/{np.max(nodes_count)}")
    print(f"Min/max bonds: {np.min(edges_count)/2:.0f}/{np.max(edges_count)/2:.0f}")

In [ ]:
# Visualization of atom and bond count distributions
if 'nodes_count' in locals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.hist(nodes_count, bins=30, alpha=0.7, color='skyblue')
    ax1.set_title('Distribution of Atom Counts')
    ax1.set_xlabel('Number of atoms')
    ax1.set_ylabel('Number of molecules')
    
    ax2.hist([e/2 for e in edges_count], bins=30, alpha=0.7, color='salmon')
    ax2.set_title('Distribution of Bond Counts')
    ax2.set_xlabel('Number of bonds')
    ax2.set_ylabel('Number of molecules')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Split into train/test (80/20)
torch.manual_seed(42)  # for reproducibility
train_len = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_len, len(dataset) - train_len]
)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Total graphs: {len(dataset)}")
print(f"Training set: {len(train_dataset)} graphs")
print(f"Test set: {len(test_dataset)} graphs")
print(f"Batch size: {train_loader.batch_size}, number of batches in training set: {len(train_loader)}")

## 4. Define GCN Model

In [ ]:
# Create a GIN model instance using our implementation from src/model.py
model = GIN(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    num_classes=dataset.num_classes,
    num_layers=2,
    dropout=0.5
).to(device)

# Base learning rate for the optimizer
base_lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=base_lr)

# Configure OneCycleLR scheduler
# total_steps - total number of training steps (epochs * batches)
num_epochs = 20
total_steps = num_epochs * len(train_loader)
scheduler = OneCycleLR(
    optimizer,
    max_lr=base_lr * 10,  # maximum learning rate (10x base rate)
    total_steps=total_steps,
    pct_start=0.3,        # percentage of steps for lr increase (30%)
    anneal_strategy='cos',  # learning rate decay strategy (cosine)
    div_factor=10.0,      # initial lr = max_lr / div_factor
    final_div_factor=1000.0  # final lr = max_lr / final_div_factor
)

print(f"Model created with {dataset.num_node_features} input features and {dataset.num_classes} output classes")
print(f"Optimizer: {optimizer.__class__.__name__} with base lr={base_lr}")
print(f"Scheduler: OneCycleLR with max_lr={base_lr * 10}")
print(model)

## 5. Training and Evaluation

In [ ]:
# Training and evaluation using our functions from src/train.py
num_epochs = 20
best_val_roc = 0
best_model_state = None
history = {'train_loss': [], 'val_roc': [], 'val_pr': [], 'lr': []}

print("Starting training with OneCycleLR scheduler...")
for epoch in range(1, num_epochs + 1):
    # Train for one epoch with scheduler
    loss, lr_history = train_epoch(model, train_loader, optimizer, device, scheduler)
    
    # Evaluate the model
    val_metrics = evaluate(model, test_loader, device)
    val_roc = val_metrics['roc_auc']
    val_pr = val_metrics['pr_auc']
    
    # Save metrics history
    history['train_loss'].append(loss)
    history['val_roc'].append(val_roc)
    history['val_pr'].append(val_pr)
    if lr_history:
        history['lr'].extend(lr_history)
    
    # Print progress with current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch:02d}: Loss = {loss:.4f}, ROC AUC = {val_roc:.4f}, PR AUC = {val_pr:.4f}, LR = {current_lr:.6f}")
    
    # Save best model state
    if val_roc > best_val_roc:
        best_val_roc = val_roc
        best_model_state = model.state_dict().copy()
        print(f"  → New best model saved! ROC AUC: {best_val_roc:.4f}")

# Load the best model state for final evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"Loaded best model with validation ROC AUC: {best_val_roc:.4f}")

# Perform final evaluation
final_metrics = evaluate(model, test_loader, device)
print("\nFinal Evaluation Results:")
print(f"Test ROC AUC: {final_metrics['roc_auc']:.4f}")
print(f"Test PR AUC: {final_metrics['pr_auc']:.4f}")

# Save the best model
torch.save(best_model_state, 'best_gin_model.pt')

## 6. Results Visualization

In [ ]:
# Visualize learning curves
plt.figure(figsize=(12, 4))

# Plot training loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(history['train_loss'])+1), history['train_loss'], 'o-', label='Training Loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plot validation ROC AUC
plt.subplot(1, 2, 2)
plt.plot(range(1, len(history['val_roc'])+1), history['val_roc'], 'o-', label='ROC AUC')
plt.plot(range(1, len(history['val_pr'])+1), history['val_pr'], 'o-', label='PR AUC')
plt.title('Validation Metrics per Epoch')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Enhanced Model Evaluation and Visualization
model.eval()

# Collect predictions for each task
all_true = {}
all_pred = {}
all_batch_losses = []

# Initialize loss function - binary cross entropy with reduction='none' to get per-task losses
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='none')

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch)  # Raw logits (before sigmoid)
        out_probs = torch.sigmoid(out).detach().cpu()
        mask = batch.y_mask.cpu()
        y = batch.y.cpu()
        
        # Calculate batch losses per task
        batch_loss = loss_fn(out, batch.y)
        masked_loss = batch_loss * batch.y_mask
        all_batch_losses.append(masked_loss.cpu())
        
        for task in range(out_probs.size(1)):
            # Select only valid entries (where mask is True)
            idx = mask[:, task]
            if idx.sum() == 0:
                continue
                
            true = y[idx, task].numpy()
            pred = out_probs[idx, task].numpy()
            
            all_true.setdefault(task, []).append(true)
            all_pred.setdefault(task, []).append(pred)

# Calculate training loss per task
all_losses = torch.cat(all_batch_losses, dim=0)  # Combine all batch losses
valid_samples_per_task = all_losses.ne(0).sum(dim=0).float()
task_losses = all_losses.sum(dim=0) / valid_samples_per_task

# Calculate AUC for each task
task_metrics = {}
for task in all_true:
    t = np.concatenate(all_true[task])
    p = np.concatenate(all_pred[task])
    
    # Skip if only one class is present
    if len(np.unique(t)) < 2:
        continue
        
    # Calculate ROC AUC
    roc_auc = roc_auc_score(t, p)
    
    # Calculate PR AUC
    prec, rec, _ = precision_recall_curve(t, p)
    pr_auc = auc(rec, prec)
    
    # Store task metrics and raw predictions for plotting
    task_metrics[task] = {
        'roc_auc': roc_auc, 
        'pr_auc': pr_auc,
        'true': t,
        'pred': p,
        'loss': task_losses[task].item() if task < len(task_losses) else float('nan')
    }

In [ ]:
# 1. Train vs Val Loss and ROC AUC plots
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Train vs Val Loss
axes[0].plot(range(1, len(history['train_loss'])+1), history['train_loss'], 'o-', label='Training Loss', color='blue')
axes[0].set_title('Train vs Validation Loss', fontsize=14)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=11)

# Train vs Val ROC AUC
axes[1].plot(range(1, len(history['val_roc'])+1), history['val_roc'], 'o-', label='Validation ROC AUC', color='green')
axes[1].set_title('Validation ROC AUC per Epoch', fontsize=14)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('ROC AUC', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0.5, 1.0])  # Set y-axis from 0.5 (random) to 1.0 (perfect)
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# 2. AUC per task (Bar plot) - Shows which tasks the model performs better/worse on
plt.figure(figsize=(14, 6))

# Extract tasks and their AUC values
tasks = list(task_metrics.keys())
roc_aucs = [task_metrics[t]['roc_auc'] for t in tasks]
pr_aucs = [task_metrics[t]['pr_auc'] for t in tasks]

# Sort by ROC AUC
sorted_indices = np.argsort(roc_aucs)
sorted_tasks = [tasks[i] for i in sorted_indices]
sorted_roc_aucs = [roc_aucs[i] for i in sorted_indices]
sorted_pr_aucs = [pr_aucs[i] for i in sorted_indices]

# Create a bar chart for both ROC AUC and PR AUC
x = np.arange(len(sorted_tasks))
width = 0.35

plt.bar(x - width/2, sorted_roc_aucs, width, label='ROC AUC', color='skyblue', alpha=0.8)
plt.bar(x + width/2, sorted_pr_aucs, width, label='PR AUC', color='salmon', alpha=0.8)

plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Random Classifier (ROC AUC=0.5)')
plt.xlabel('Task Index', fontsize=12)
plt.ylabel('AUC Score', fontsize=12)
plt.title('Model Performance (AUC) per Task', fontsize=14)
plt.xticks(x, [f'{t}' for t in sorted_tasks], rotation=90)
plt.legend(fontsize=11)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 3. ROC and PR curves for selected tasks
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Find tasks with good, medium, and lower performance (for more diverse visualization)
roc_auc_values = np.array([task_metrics[t]['roc_auc'] for t in task_metrics])
best_idx = np.argmax(roc_auc_values)
worst_idx = np.argmin(roc_auc_values)
median_idx = np.argsort(roc_auc_values)[len(roc_auc_values)//2]

selected_tasks = [list(task_metrics.keys())[i] for i in [best_idx, median_idx, worst_idx]]
colors = ['green', 'blue', 'orange']

# ROC Curves
for i, task in enumerate(selected_tasks):
    t = task_metrics[task]['true']
    p = task_metrics[task]['pred']
    fpr, tpr, _ = roc_curve(t, p)
    axes[0].plot(fpr, tpr, color=colors[i], 
               label=f'Task {task} (AUC = {task_metrics[task]["roc_auc"]:.3f})')

axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.4, label='Random (AUC = 0.5)')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curves', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# PR Curves
for i, task in enumerate(selected_tasks):
    t = task_metrics[task]['true']
    p = task_metrics[task]['pred']
    prec, rec, _ = precision_recall_curve(t, p)
    axes[1].plot(rec, prec, color=colors[i],
               label=f'Task {task} (AUC = {task_metrics[task]["pr_auc"]:.3f})')

# Calculate and plot baseline for PR curve (class imbalance)
for task in selected_tasks:
    t = task_metrics[task]['true']
    baseline = np.sum(t) / len(t)  # Positive class ratio
    axes[1].axhline(y=baseline, color=colors[selected_tasks.index(task)], 
                  linestyle='--', alpha=0.4, 
                  label=f'Task {task} baseline: {baseline:.3f}')

axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curves', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 4. Prediction histograms - Shows class separability
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Prediction Distribution by Class', fontsize=16, y=1.02)

for i, task in enumerate(selected_tasks):
    t = task_metrics[task]['true']
    p = task_metrics[task]['pred']
    
    # Split predictions by true class
    pos_preds = p[t == 1]
    neg_preds = p[t == 0]
    
    # Determine class balance percentages
    pos_perc = 100 * len(pos_preds) / len(t)
    neg_perc = 100 * len(neg_preds) / len(t)
    
    # Create the histogram
    axes[i].hist(neg_preds, bins=20, alpha=0.6, color='red', label=f'Negative ({neg_perc:.1f}%)', density=True)
    axes[i].hist(pos_preds, bins=20, alpha=0.6, color='blue', label=f'Positive ({pos_perc:.1f}%)', density=True)
    
    # Calculate and display separability metrics
    mean_diff = np.mean(pos_preds) - np.mean(neg_preds) if len(pos_preds) > 0 and len(neg_preds) > 0 else 0
    roc_auc = task_metrics[task]['roc_auc']
    
    axes[i].set_title(f'Task {task}\nROC AUC: {roc_auc:.3f}, Mean Diff: {mean_diff:.3f}', fontsize=12)
    axes[i].set_xlabel('Prediction Probability', fontsize=11)
    axes[i].set_ylabel('Density', fontsize=11)
    axes[i].grid(True, alpha=0.3)
    axes[i].legend(fontsize=9)
    axes[i].set_xlim([0, 1])

    # Add a vertical line for the optimal threshold (using Youden's J statistic)
    if len(pos_preds) > 0 and len(neg_preds) > 0:
        fpr, tpr, thresholds = roc_curve(t, p)
        # Calculate Youden's J statistic: J = sensitivity + specificity - 1
        j_scores = tpr - fpr
        # Get the optimal threshold
        best_idx = np.argmax(j_scores)
        best_threshold = thresholds[best_idx]
        axes[i].axvline(x=best_threshold, color='green', linestyle='--', alpha=0.8, 
                      label=f'Optimal threshold: {best_threshold:.2f}')
        axes[i].legend(fontsize=9)
    
plt.tight_layout()
plt.show()

In [ ]:
# 5. Combined loss and performance visualization
plt.figure(figsize=(12, 6))

# Extract task losses and ROC AUCs
tasks = list(task_metrics.keys())
losses = [task_metrics[t]['loss'] for t in tasks]
roc_aucs = [task_metrics[t]['roc_auc'] for t in tasks]

# Sort by loss
sorted_indices = np.argsort(losses)
sorted_tasks = [tasks[i] for i in sorted_indices]
sorted_losses = [losses[i] for i in sorted_indices]
sorted_roc_aucs = [roc_aucs[i] for i in sorted_indices]

# Create two y-axes for loss and AUC
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot loss bars
bar_colors = ['#FF9999' if auc < 0.7 else '#66B2FF' for auc in sorted_roc_aucs]
bars = ax1.bar(sorted_tasks, sorted_losses, alpha=0.7, color=bar_colors)

# Create a second y-axis for AUC
ax2 = ax1.twinx()
ax2.plot(sorted_tasks, sorted_roc_aucs, 'go-', alpha=0.7, label='ROC AUC')
ax2.set_ylim([0.4, 1.0])

# Add labels and title
ax1.set_xlabel('Task', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12, color='red')
ax2.set_ylabel('ROC AUC', fontsize=12, color='green')
plt.title('Task-wise Loss vs Performance', fontsize=14)

# Add gridlines
ax1.grid(True, alpha=0.3)

# Add explanatory threshold line
ax2.axhline(y=0.7, color='green', linestyle='--', alpha=0.5, label='AUC = 0.7')

# Add legends
ax1.legend(handles=[
    plt.Rectangle((0,0), 1, 1, color='#FF9999', alpha=0.7, label='AUC < 0.7'),
    plt.Rectangle((0,0), 1, 1, color='#66B2FF', alpha=0.7, label='AUC ≥ 0.7')
], loc='upper left')

ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 7. Conclusion and Findings

# TODO: Add analysis of results and conclusions